In [1]:
# initial imports and settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sc2reader
import re
import random
import datetime
import json
from scripts.classes import ReplayInfo
import time
import math

# sklearn
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

# import random forest
from sklearn.ensemble import RandomForestClassifier

# import KNN
from sklearn.neighbors import KNeighborsClassifier

# import grid_search
from sklearn.model_selection import GridSearchCV

# import warnings
import warnings

In [3]:
replays_dir = "data/events"
columns_list = []
index_list = []

start_time = time.time()
total_replays = len(os.listdir(replays_dir))
# loop through each pkl in the directory
for i, filename in enumerate(os.listdir(replays_dir)):
   
    # print progress and eta
    eta = round((time.time() - start_time)/(i+1)*(total_replays-i),0)
    print(
        f'\r{i+1}/{total_replays} - ETA: {datetime.timedelta(seconds=eta)}',
        end=''
    )

    # handle both pkl and csv files
    if filename.endswith(".pkl"):
        # load the pkl file into a dataframe
        df = pd.read_pickle(os.path.join(replays_dir, filename))
    elif filename.endswith(".csv"):
        # load the csv file into a dataframe
        df = pd.read_csv(os.path.join(replays_dir, filename), index_col=0)
    else:
        continue

    # remove extension from filename
    filename_no_ext = filename.split(".")[0]
    index_list.append(filename_no_ext)

    # get the column names
    columns_list.append(df.columns.values)


36812/36812 - ETA: 0:00:00

In [2]:
# create a dataframe with all the column names
columns_df = pd.DataFrame(
    [sorted(x) for x in columns_list], 
    index=index_list
)
columns_df.head()


NameError: name 'columns_list' is not defined

In [5]:
# find any rows that have nan values
columns_df[columns_df.isnull().any(axis=1)]


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
00963846e01f659436d5ffdd92482740d9258b88c546a2ed84e85699dd7c5553,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,frame,...,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None,None
00d98eeeb97d26e05e1fb403bbe6007a017791d5f56167915811e4ad86209601,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,frame,...,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None,None
014c724a43f733c4fcfc2fcf3b130c1bc7bcc857ecbf80b232c71dcef624ac6d,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,frame,...,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None,None
019ca1cc8f0dbecb198105696c1a0a8c338e95d023410acdf7ecfa6437e65e50,ability_id,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,...,vespene_used_current,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None
02ddc3ab01147d478733579be09edae2dcceb5458906422da0e2b8cf907ca3b4,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,frame,...,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fdfb3eb49acae60a8f5b4ed648797de94788ad60566bd6f14f07a93b47ec57bb,ability_id,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,...,vespene_used_current,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None
fe47e889289211a435b8e893945fdf86df57d5b97fa729f2f97a296a29c6dae5,ability_id,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,...,vespene_used_current,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,workers_active_count,None
fe8691dd94c3621b179a504232a5ea7698d522f55f18d2a8637313005291d2ee,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,food_made,food_used,frame,...,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,vespene_used_in_progress,vesp

In [10]:

df = pd.read_pickle('data/events/0b602a51c350d3969406bc48230714e3cdfae1e7ee9bd3e885ba15684f96295f.pkl')
# pd.get_dummies(df['name'])

In [11]:
df.set_index('frame', inplace=True)
df.info()

frame_range = range(max(df.index)+1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5143 entries, 0 to 13181
Data columns (total 59 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   name                                  5143 non-null   object 
 1   pid                                   4348 non-null   float64
 2   second                                5143 non-null   int64  
 3   count                                 21 non-null     float64
 4   upgrade_type_name                     21 non-null     object 
 5   unit_type_name                        494 non-null    object 
 6   ff_minerals_lost_army                 169 non-null    float64
 7   ff_minerals_lost_economy              169 non-null    float64
 8   ff_minerals_lost_technology           169 non-null    float64
 9   ff_vespene_lost_army                  169 non-null    float64
 10  ff_vespene_lost_economy               169 non-null    float64
 11  ff_vespene_lost_

In [12]:
df.shape

(5143, 59)

In [13]:
# list of unnecessary_event
unnecessary_events = [
    'AddToControlGroupEvent',
    'CameraEvent',
    'ChatEvent',
    'GetControlGroupEvent',
    'PlayerSetupEvent',
    'PlayerLeaveEvent',
    'ProgressEvent',
    'SelectionEvent',
    'SetControlGroupEvent',
    'UnitPositionsEvent',
    'UnitTypeChangeEvent',
    'UpdateTargetPointCommandEvent',
    'UpdateTargetUnitCommandEvent',
    'UserOptionsEvent'
]

necessary_events = [
    'TargetPointCommandEvent',
    'UnitBornEvent',
    'PlayerStatsEvent',
    'UnitDiedEvent',
    'UnitInitEvent',
    'UnitDoneEvent',
    'TargetUnitCommandEvent',
    'BasicCommandEvent',
    'UpgradeCompleteEvent'
]

# remove all rows where 'name' is in the unnecessary_events list
df = df[df['name'].isin(necessary_events)]
df.shape

(1450, 59)

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
pid,707.0,1.865629,0.341291,1.0,2.0,2.0,2.0,2.0
second,1450.0,422.540690,273.407024,0.0,175.0,461.0,660.0,823.0
count,21.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
ff_minerals_lost_army,169.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ff_minerals_lost_economy,169.0,46.745562,82.401097,0.0,0.0,0.0,100.0,200.0
ff_minerals_lost_technology,169.0,106.508876,198.248885,0.0,0.0,0.0,100.0,550.0
ff_vespene_lost_army,169.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ff_vespene_lost_economy,169.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ff_vespene_lost_technology,169.0,23.964497,55.121283,0.0,0.0,0.0,0.0,150.0
food_made,169.0,65.538462,34.798741,15.0,31.0,62.0,102.0,117.0


In [15]:
unnecessary_columns = [
    'count',
    'second',
    'killer_pid',
    'ff_minerals_lost_army',
    'ff_minerals_lost_economy',
    'ff_minerals_lost_technology',
    'ff_vespene_lost_army',
    'ff_vespene_lost_economy',
    'ff_vespene_lost_technology'
]
# drop unnecessary columns
df.drop(columns=unnecessary_columns, inplace=True)


    

In [16]:
unnecessary_upgrades = set(
    [x for x in df['upgrade_type_name'] if 'spray' in str(x).lower()]
)

for x in unnecessary_upgrades:
    df = df[df['upgrade_type_name'] != x]

df.shape

(1437, 50)

In [17]:
df = pd.get_dummies(df, columns=['name', 'upgrade_type_name'])


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1437 entries, 0 to 13181
Data columns (total 62 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   pid                                           694 non-null    float64
 1   unit_type_name                                476 non-null    object 
 2   food_made                                     169 non-null    float64
 3   food_used                                     169 non-null    float64
 4   minerals_collection_rate                      169 non-null    float64
 5   minerals_current                              169 non-null    float64
 6   minerals_killed                               169 non-null    float64
 7   minerals_killed_army                          169 non-null    float64
 8   minerals_killed_economy                       169 non-null    float64
 9   minerals_killed_technology                    169 non-null    

In [19]:
player1_df = df[df['pid'] == 1].copy()
player2_df = df[df['pid'] == 2].copy()

# drop pid column
player1_df.drop(columns=['pid'], inplace=True)
player2_df.drop(columns=['pid'], inplace=True)
# drop 'unit_type_name' column from player1_df and player2_df
player1_df.drop(columns=['unit_type_name'], inplace=True)
player2_df.drop(columns=['unit_type_name'], inplace=True)

# drop all rows where 'pid' is 1 or 2 from df
df = df[df['pid'] != 1]
df = df[df['pid'] != 2]

print(player1_df.shape, player2_df.shape, df.shape)

(88, 60) (606, 60) (743, 62)


In [20]:
# drop all null columns from df
df.dropna(axis=1, how='all', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 0 to 13181
Data columns (total 15 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   unit_type_name                                476 non-null    object
 1   name_BasicCommandEvent                        743 non-null    uint8 
 2   name_PlayerStatsEvent                         743 non-null    uint8 
 3   name_TargetPointCommandEvent                  743 non-null    uint8 
 4   name_TargetUnitCommandEvent                   743 non-null    uint8 
 5   name_UnitBornEvent                            743 non-null    uint8 
 6   name_UnitDiedEvent                            743 non-null    uint8 
 7   name_UnitDoneEvent                            743 non-null    uint8 
 8   name_UnitInitEvent                            743 non-null    uint8 
 9   name_UpgradeCompleteEvent                     743 non-null    uint8 
 10  

In [21]:
# get all numeric columns in df
numeric_columns = [x for x in df.columns if df[x].dtype != 'object']

# get all numeric columns where sum is 0 in df
zero_columns = [x for x in numeric_columns if df[x].sum() == 0]

df.drop(columns=zero_columns, inplace=True)

In [22]:

# convert all values in df['unit_type_name'] to strings
df['unit_type_name'] = df['unit_type_name'].astype(str)
df['unit_type_name'].unique()

array(['LabMineralField', 'RichMineralField', 'LabMineralField750',
       'SpacePlatformGeyser', 'InhibitorZoneSmall', 'RichMineralField750',
       'UnbuildablePlatesDestructible', 'DestructibleDebris6x6',
       'DestructibleDebrisRampDiagonalHugeBLUR',
       'DestructibleDebrisRampDiagonalHugeULBR', 'BeaconArmy',
       'BeaconDefend', 'BeaconAttack', 'BeaconHarass', 'BeaconIdle',
       'BeaconAuto', 'BeaconDetect', 'BeaconScout', 'BeaconClaim',
       'BeaconExpand', 'BeaconRally', 'BeaconCustom1', 'BeaconCustom2',
       'BeaconCustom3', 'BeaconCustom4', 'Nexus', 'Probe', 'Pylon', 'nan',
       'Gateway', 'Assimilator', 'CyberneticsCore', 'Zealot', 'Stargate',
       'Stalker', 'ShieldBattery', 'Adept', 'AdeptPhaseShift', 'VoidRay',
       'Phoenix', 'TwilightCouncil', 'Oracle', 'DarkShrine',
       'DarkTemplar', 'Forge', 'RoboticsFacility', 'Sentry', 'ForceField',
       'Archon', 'Observer', 'Immortal'], dtype=object)

In [23]:


unique_units = df['unit_type_name'].unique()
unnecessary_units =[]
for unit in unique_units:
    if ('beacon' in unit.lower()) or ('nan' in unit.lower()):
        unnecessary_units.append(unit)

# drop all rows in df where 'unit_type_name' is in unnecessary_units
df = df[~df['unit_type_name'].isin(unnecessary_units)]

# dummify unit_type_name
df = pd.get_dummies(df, columns=['unit_type_name'])

df.info()        


<class 'pandas.core.frame.DataFrame'>
Int64Index: 446 entries, 0 to 13134
Data columns (total 39 columns):
 #   Column                                                 Non-Null Count  Dtype
---  ------                                                 --------------  -----
 0   name_UnitBornEvent                                     446 non-null    uint8
 1   name_UnitDiedEvent                                     446 non-null    uint8
 2   name_UnitDoneEvent                                     446 non-null    uint8
 3   name_UnitInitEvent                                     446 non-null    uint8
 4   unit_type_name_Adept                                   446 non-null    uint8
 5   unit_type_name_AdeptPhaseShift                         446 non-null    uint8
 6   unit_type_name_Archon                                  446 non-null    uint8
 7   unit_type_name_Assimilator                             446 non-null    uint8
 8   unit_type_name_CyberneticsCore                         446 non-null 

In [24]:
# rename columns to player_1_ and player_2_
player1_df.columns = ['player_1_' + x for x in player1_df.columns]
player2_df.columns = ['player_2_' + x for x in player2_df.columns]


In [25]:

print(player1_df.shape, player2_df.shape, df.shape)

# drop duplicate rows
player1_df.drop_duplicates(inplace=True)
player2_df.drop_duplicates(inplace=True)

# for all missing value rows in player1_df and player2_df, fill with above row
player1_df.fillna(method='ffill', inplace=True)
player2_df.fillna(method='ffill', inplace=True)

print(player1_df.shape, player2_df.shape, df.shape)

# drop duplicate rows again
player1_df.drop_duplicates(inplace=True)
player2_df.drop_duplicates(inplace=True)

print(player1_df.shape, player2_df.shape, df.shape)

(88, 60) (606, 60) (446, 39)
(87, 60) (134, 60) (446, 39)
(87, 60) (134, 60) (446, 39)


In [26]:
# make new df with index of frame_range
new_df = pd.DataFrame(index=frame_range)
new_df

""
0
1
2
3
4
...
13177
13178
13179
13180


In [36]:
df.loc[282, ['name_UnitBornEvent', 'unit_type_name_Probe']]

,name_UnitBornEvent,unit_type_name_Probe
frame,,
282,1,1
282,1,1


In [35]:
# find all columns with values not ==0 at index 282
for col in df.columns:
    if df.loc[282, col].sum() != 0:
        print(col)

name_UnitBornEvent
unit_type_name_Probe


In [27]:
new_df = new_df.join(player1_df)
new_df = new_df.join(player2_df)
new_df = new_df.join(df)

In [28]:
# drop all rows with index = 0
new_df.drop(index=0, inplace=True)

# insert single row with index = 0 with all 0 values
new_df.loc[0] = 0

# create new row with index = 0 backfilled with first non-nan row
# for each column get first non-nan value
for col in new_df.columns:
    first_non_nan = new_df[col].first_valid_index()
    new_df.loc[0, col] = new_df.loc[first_non_nan, col]


# sort new_df by index
new_df.sort_index(inplace=True)
new_df

ValueError: Incompatible indexer with Series

In [40]:
# get all columns from player1_df and player2_df
player_columns_mask = player1_df.columns.tolist() + player2_df.columns.tolist()

# forward fill all new_df mask columns 
new_df[player_columns_mask] = new_df[player_columns_mask].fillna(method='ffill').copy()

new_df

,player_1_food_made,player_1_food_used,player_1_minerals_collection_rate,player_1_minerals_current,player_1_minerals_killed,player_1_minerals_killed_army,player_1_minerals_killed_economy,player_1_minerals_killed_technology,player_1_minerals_lost,player_1_minerals_lost_army,...,unit_type_name_Sentry,unit_type_name_ShieldBattery,unit_type_name_SpacePlatformGeyser,unit_type_name_Stalker,unit_type_name_TemplarArchive,unit_type_name_TwilightCouncil,unit_type_name_UnbuildableRocksDestructible,unit_type_name_VespeneGeyser,unit_type_name_WarpPrism,unit_type_name_Zealot
0,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15176,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15177,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15178,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15179,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# fillna for all other columns
new_df.fillna(0, inplace=True)
new_df

,player_1_food_made,player_1_food_used,player_1_minerals_collection_rate,player_1_minerals_current,player_1_minerals_killed,player_1_minerals_killed_army,player_1_minerals_killed_economy,player_1_minerals_killed_technology,player_1_minerals_lost,player_1_minerals_lost_army,...,unit_type_name_Sentry,unit_type_name_ShieldBattery,unit_type_name_SpacePlatformGeyser,unit_type_name_Stalker,unit_type_name_TemplarArchive,unit_type_name_TwilightCouncil,unit_type_name_UnbuildableRocksDestructible,unit_type_name_VespeneGeyser,unit_type_name_WarpPrism,unit_type_name_Zealot
0,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.0,12.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15176,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15177,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15178,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15179,103.0,51.0,531.0,360.0,3800.0,2400.0,900.0,500.0,7975.0,5600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
new_df.isna().sum().sum()

0

In [43]:
new_df['game_winner']

KeyError: 'game_winner'

In [17]:
replay_df = pd.read_csv('data/replays.csv')
replay_df.head()

,filename,map,player1_race,player2_race,player1_mmr,player2_mmr,game_length,game_type,game_speed,game_winner,timestamp,fps,is_ladder,region,player1_highest_league,player2_highest_league,filehash
0,C:/Users/jared/Gits/sc2-modeling/data/Spawning...,abb20e9c587f5f6ce279a6aa8c7e9be2897607abb00434...,Terran,Protoss,-36400.0,NaN,402,1v1,Normal,1,1633598712,16.0,False,us,4,0,9384fda8c370ea7d130ac20244f7d0fda9a9b834004445...
1,C:/Users/jared/Gits/sc2-modeling/data/Spawning...,abb20e9c587f5f6ce279a6aa8c7e9be2897607abb00434...,Terran,Zerg,-36400.0,NaN,371,1v1,Normal,1,1633603684,16.0,False,us,4,0,f9864054498acf297aacf2be80896ba131716a341983de...
2,C:/Users/jared/Gits/sc2-modeling/data/Spawning...,c2bcc137d9ce43fbf9a5ddd04f5167892c779c29616de9...,Zerg,Zerg,4022.0,3502.0,371,1v1,Faster,1,1633788999,16.0,True,eu,6,6,178a82fa5045e82a3920688274d7a524595f0ef6dc91ae...
3,C:/Users/jared/Gits/sc2-modeling/data/Spawning...,e83bd9ddb33fcdc21a801f9bd598ed081f8ca60f810522...,Zerg,Protoss,NaN,NaN,1803,1v1,Faster,1,1630608640,16.0,False,eu,7,6,99ba80721116d18f965c141581baa878abf02526f48bc6...
4,C:/Users/jared/Gits/sc2-modeling/data/Spawning...,e83bd9ddb33fcdc21a801f9bd598ed081f8ca60f810522...,Zerg,Protoss,3615.0,3567.0,502,1v1,Faster,1,1633138764,16.0,True,us,5,5,b9e96a1dabcc0de0cfd89e8f0d02b5b8e064ec3f3c07c1...


In [7]:
a = pd.DataFrame({'a':[1, 2, 3], 'b':[np.nan,np.nan,np.nan]})
frst = a['b'].first_valid_index()

In [9]:
print(frst)

None


In [20]:
replay_df['game_winner'].value_counts()

1    18729
2    16024
0     2059
Name: game_winner, dtype: int64